# Step2.1 首先执行下面这个：定义函数

In [1]:
########################################################
# Author:                   
# Date: Nov.8, 2018         
# Version: v3            
# Notice:      
# 1. FeatureExtract
# 2. directories structure
#   code/
#   model/
#   feature/
#   answer/
#   train/
#   A/
#   MAKE SURE ALL OF THE DIR IS EXISTED!!!
# 3. 特征均存储在../feature目录
# 
# 4. 最终的特征汇总在features_all_train_test_20181106.csv
#    包含所有train和A的数据
#########################################################


import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

# common prefix
TRAIN_PREFIX = "../train/"
TEST_PREFIX = "../B/"
FEATURE_PREFIX = "../feature/"

# File Encoder
FILE_ENCODER = "utf-8"

#Train-dataset
BANK_DETAIL_TRAIN_FILE = TRAIN_PREFIX + "bank_detail_train.csv"
BILL_DETAIL_TRAIN_FILE = TRAIN_PREFIX + "bill_detail_train.csv"
BROWSE_HISTORY_TRAIN_FILE = TRAIN_PREFIX +"browse_history_train.csv"
LOANTIME_TRAIN_FILE = TRAIN_PREFIX + "loantime_train.csv"
OVERDUE_TRAIN_FILE = TRAIN_PREFIX + "overdue_train.csv"
USERINFO_TRAIN_FILE = TRAIN_PREFIX + "userinfo_train.csv"

#Test-dataset
BANK_DETAIL_TEST_FILE = TEST_PREFIX + "bank_detail_B.csv"
BILL_DETAIL_TEST_FILE = TEST_PREFIX + "bill_detail_B.csv"
BROWSE_HISTORY_TEST_FILE = TEST_PREFIX + "browse_history_B.csv"
LOANTIME_TEST_FILE = TEST_PREFIX + "loantime_B.csv"
USERINFO_TEST_FILE = TEST_PREFIX + "userinfo_B.csv"
USERID_TO_PREDICT_FILE = FEATURE_PREFIX + "to_predict_userid.csv" #因为A目录没有写入权限   


In [2]:
# 增加放款给用户之前，用户信用卡账单的各种统计信息sum count max min mean std var等
def buildFeatureBeforeLoan(d, feature):
    prefix = '放款前'
    condition = (d['时间']<=d['放款时间'])
    #print(d[condition].shape)
    group_by=d[condition].loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                      '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],
                                                                                                     as_index=False)
    agg_param = ['sum', 'count', 'max', 'min', 'median', 'mean', 'std', 'var']
    bill_beforeloan_agg = group_by.agg(agg_param)
    bill_beforeloan_agg.columns = ['_'.join(x) for x in bill_beforeloan_agg.columns.ravel()]
    bill_beforeloan_agg = bill_beforeloan_agg.add_prefix(prefix).reset_index()
    feature=pd.merge(feature, bill_beforeloan_agg,how='left', on = "用户标识")
    #print(feature.shape)
    for stat in agg_param:
        feature[prefix+'上期还款金额'+stat+'与'+prefix+'上期账单金额'+stat+'差值']=feature[prefix+'上期还款金额_'+stat]-feature[prefix+'上期账单金额_'+stat]
        feature[prefix+'信用卡额度'+stat+'与'+prefix+'本期账单余额'+stat+'差值']=feature[prefix+'信用卡额度_'+stat]-feature[prefix+'本期账单余额_'+stat]
        feature[prefix+'可用余额'+stat+'与'+prefix+'预借现金额度'+stat+'差值']=feature[prefix+'可用余额_'+stat]-feature[prefix+'预借现金额度_'+stat]
        feature[prefix+'本期账单最低还款额'+stat+'与'+prefix+'循环利息'+stat+'之和']=feature[prefix+'本期账单最低还款额_'+stat]+feature[prefix+'循环利息_'+stat]
    #print(feature.shape)
    
    feature_basic = d[condition]
    feature_basic['相邻两期账单金额差'] = feature_basic['本期账单金额'] - feature_basic['上期账单金额']
    feature_basic['本期还款总额'] = feature_basic['上期账单金额'] - feature_basic['上期还款金额'] + feature_basic['本期账单金额'] - feature_basic['调整金额'] + feature_basic['循环利息']
    feature_basic['已经使用信用卡额度'] = feature_basic['信用卡额度'] - feature_basic['可用余额']
    gb = feature_basic.loc[:,['用户标识' ,'相邻两期账单金额差', '本期还款总额', '已经使用信用卡额度']].groupby(["用户标识"], as_index=False)
    feature_agg = gb.agg({'mean', 'var', 'max', 'min', 'median'}).add_prefix(prefix+"_").reset_index()
    feature=pd.merge(feature, feature_agg,how='left', on = "用户标识")
    #print(feature.shape)
    # END
    return feature

In [3]:
#放款前账单去重统计特征
def buildFeatureBeforeLoan2(d, feature):
    prefix = '去重后放款前'
    condition = (d['时间']<=d['放款时间'])
    #print(d[condition].shape)
    data=d[condition].loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
                                      ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
                                      ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()
    group_by=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                   '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)
    
    agg_param = ['sum', 'count', 'max', 'min', 'median', 'mean', 'std', 'var']
    bill_beforeloan_agg = group_by.agg(agg_param)
    bill_beforeloan_agg.columns = ['_'.join(x) for x in bill_beforeloan_agg.columns.ravel()]
    bill_beforeloan_agg = bill_beforeloan_agg.add_prefix(prefix).reset_index()
    feature=pd.merge(feature, bill_beforeloan_agg,how='left', on = "用户标识")
    #print(feature.shape)
    for stat in agg_param:
        feature[prefix+'上期还款金额'+stat+'与'+prefix+'上期账单金额'+stat+'差值']=feature[prefix+'上期还款金额_'+stat]-feature[prefix+'上期账单金额_'+stat]
        feature[prefix+'信用卡额度'+stat+'与'+prefix+'本期账单余额'+stat+'差值']=feature[prefix+'信用卡额度_'+stat]-feature[prefix+'本期账单余额_'+stat]
        feature[prefix+'可用余额'+stat+'与'+prefix+'预借现金额度'+stat+'差值']=feature[prefix+'可用余额_'+stat]-feature[prefix+'预借现金额度_'+stat]
        feature[prefix+'本期账单最低还款额'+stat+'与'+prefix+'循环利息'+stat+'之和']=feature[prefix+'本期账单最低还款额_'+stat]+feature[prefix+'循环利息_'+stat]
    #print(feature.shape)
    return feature#55596 rows × 242 columns

In [4]:
# 增加 放款后的各种统计信息
def buildFeatureAfterLoan(d, feature):
    prefix = '放款后'
    condition = (d['时间']>d['放款时间'])
    #print(d[condition].shape)
    group_by=d[condition].loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                      '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],
                                                                                                     as_index=False)
    agg_param = ['sum', 'count', 'max', 'min', 'median', 'mean', 'std', 'var']
    bill_beforeloan_agg = group_by.agg(agg_param)
    bill_beforeloan_agg.columns = ['_'.join(x) for x in bill_beforeloan_agg.columns.ravel()]
    bill_beforeloan_agg = bill_beforeloan_agg.add_prefix(prefix).reset_index()
    feature=pd.merge(feature, bill_beforeloan_agg,how='left', on = "用户标识")
    #print(feature.shape)
    for stat in agg_param:
        feature[prefix+'上期还款金额'+stat+'与'+prefix+'上期账单金额'+stat+'差值']=feature[prefix+'上期还款金额_'+stat]-feature[prefix+'上期账单金额_'+stat]
        feature[prefix+'信用卡额度'+stat+'与'+prefix+'本期账单余额'+stat+'差值']=feature[prefix+'信用卡额度_'+stat]-feature[prefix+'本期账单余额_'+stat]
        feature[prefix+'可用余额'+stat+'与'+prefix+'预借现金额度'+stat+'差值']=feature[prefix+'可用余额_'+stat]-feature[prefix+'预借现金额度_'+stat]
        feature[prefix+'本期账单最低还款额'+stat+'与'+prefix+'循环利息'+stat+'之和']=feature[prefix+'本期账单最低还款额_'+stat]+feature[prefix+'循环利息_'+stat]
    #print(feature.shape)
    
    feature_basic = d[condition]
    feature_basic['相邻两期账单金额差'] = feature_basic['本期账单金额'] - feature_basic['上期账单金额']
    feature_basic['本期还款总额'] = feature_basic['上期账单金额'] - feature_basic['上期还款金额'] + feature_basic['本期账单金额'] - feature_basic['调整金额'] + feature_basic['循环利息']
    feature_basic['已经使用信用卡额度'] = feature_basic['信用卡额度'] - feature_basic['可用余额']
    gb = feature_basic.loc[:,['用户标识' ,'相邻两期账单金额差', '本期还款总额', '已经使用信用卡额度']].groupby(["用户标识"], as_index=False)
    feature_agg = gb.agg({'mean', 'var', 'max', 'min', 'median'}).add_prefix(prefix+"_").reset_index()
    feature=pd.merge(feature, feature_agg,how='left', on = "用户标识")
    #print(feature.shape)
    
    return feature

In [5]:
#放款后账单去重统计特征
def buildFeatureAfterLoan2(d, feature):
    prefix = '去重后放款后'
    condition = (d['时间']>d['放款时间'])
    #print(d[condition].shape)
    data=d[condition].loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
                                      ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
                                      ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()
    group_by=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                   '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)
    
    agg_param = ['sum', 'count', 'max', 'min', 'median', 'mean', 'std', 'var']
    bill_beforeloan_agg = group_by.agg(agg_param)
    bill_beforeloan_agg.columns = ['_'.join(x) for x in bill_beforeloan_agg.columns.ravel()]
    bill_beforeloan_agg = bill_beforeloan_agg.add_prefix(prefix).reset_index()
    feature=pd.merge(feature, bill_beforeloan_agg,how='left', on = "用户标识")
    #print(feature.shape)
    for stat in agg_param:
        feature[prefix+'上期还款金额'+stat+'与'+prefix+'上期账单金额'+stat+'差值']=feature[prefix+'上期还款金额_'+stat]-feature[prefix+'上期账单金额_'+stat]
        feature[prefix+'信用卡额度'+stat+'与'+prefix+'本期账单余额'+stat+'差值']=feature[prefix+'信用卡额度_'+stat]-feature[prefix+'本期账单余额_'+stat]
        feature[prefix+'可用余额'+stat+'与'+prefix+'预借现金额度'+stat+'差值']=feature[prefix+'可用余额_'+stat]-feature[prefix+'预借现金额度_'+stat]
        feature[prefix+'本期账单最低还款额'+stat+'与'+prefix+'循环利息'+stat+'之和']=feature[prefix+'本期账单最低还款额_'+stat]+feature[prefix+'循环利息_'+stat]
    #print(feature.shape)
    return feature#55596 rows × 242 columns

In [6]:
def buildFeatureNotime(d, feature):
    
    prefix = '时间未知'
    group_by=d.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                      '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],as_index=False)
    agg_param = ['sum', 'count', 'max', 'min', 'median', 'mean', 'std', 'var']
    bill_beforeloan_agg = group_by.agg(agg_param)
    bill_beforeloan_agg.columns = ['_'.join(x) for x in bill_beforeloan_agg.columns.ravel()]
    bill_beforeloan_agg = bill_beforeloan_agg.add_prefix(prefix).reset_index()
    feature=pd.merge(feature, bill_beforeloan_agg,how='left', on = "用户标识")
    #print(feature.shape)
    for stat in agg_param:
        feature[prefix+'上期还款金额'+stat+'与'+prefix+'上期账单金额'+stat+'差值']=feature[prefix+'上期还款金额_'+stat]-feature[prefix+'上期账单金额_'+stat]
        feature[prefix+'信用卡额度'+stat+'与'+prefix+'本期账单余额'+stat+'差值']=feature[prefix+'信用卡额度_'+stat]-feature[prefix+'本期账单余额_'+stat]
        feature[prefix+'可用余额'+stat+'与'+prefix+'预借现金额度'+stat+'差值']=feature[prefix+'可用余额_'+stat]-feature[prefix+'预借现金额度_'+stat]
        feature[prefix+'本期账单最低还款额'+stat+'与'+prefix+'循环利息'+stat+'之和']=feature[prefix+'本期账单最低还款额_'+stat]+feature[prefix+'循环利息_'+stat]
    #print(feature.shape)
    
    feature_basic = d
    feature_basic['相邻两期账单金额差'] = feature_basic['本期账单金额'] - feature_basic['上期账单金额']
    feature_basic['本期还款总额'] = feature_basic['上期账单金额'] - feature_basic['上期还款金额'] + feature_basic['本期账单金额'] - feature_basic['调整金额'] + feature_basic['循环利息']
    feature_basic['已经使用信用卡额度'] = feature_basic['信用卡额度'] - feature_basic['可用余额']
    gb = feature_basic.loc[:,['用户标识' ,'相邻两期账单金额差', '本期还款总额', '已经使用信用卡额度']].groupby(["用户标识"], as_index=False)
    feature_agg = gb.agg({'mean', 'var', 'max', 'min', 'median'}).add_prefix(prefix+"_").reset_index()
    feature=pd.merge(feature, feature_agg,how='left', on = "用户标识")
    #print(feature.shape)
    return feature#55596 rows × 242 columns


In [7]:
#train=pd.read_csv("../feature/train_20170116_A.csv",encoding="gb2312")
#时间未知账单去重统计特征
def buildFeatureNotime2(d, feature):
    prefix = '去重后时间未知'
    data=d.loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
                  ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
                  ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

    group_by=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                   '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

    agg_param = ['sum', 'count', 'max', 'min', 'median', 'mean', 'std', 'var']
    bill_beforeloan_agg = group_by.agg(agg_param)
    bill_beforeloan_agg.columns = ['_'.join(x) for x in bill_beforeloan_agg.columns.ravel()]
    bill_beforeloan_agg = bill_beforeloan_agg.add_prefix(prefix).reset_index()
    feature=pd.merge(feature, bill_beforeloan_agg,how='left', on = "用户标识")
    #print(feature.shape)
    for stat in agg_param:
        feature[prefix+'上期还款金额'+stat+'与'+prefix+'上期账单金额'+stat+'差值']=feature[prefix+'上期还款金额_'+stat]-feature[prefix+'上期账单金额_'+stat]
        feature[prefix+'信用卡额度'+stat+'与'+prefix+'本期账单余额'+stat+'差值']=feature[prefix+'信用卡额度_'+stat]-feature[prefix+'本期账单余额_'+stat]
        feature[prefix+'可用余额'+stat+'与'+prefix+'预借现金额度'+stat+'差值']=feature[prefix+'可用余额_'+stat]-feature[prefix+'预借现金额度_'+stat]
        feature[prefix+'本期账单最低还款额'+stat+'与'+prefix+'循环利息'+stat+'之和']=feature[prefix+'本期账单最低还款额_'+stat]+feature[prefix+'循环利息_'+stat]
    #print(feature.shape)
    return feature#55596 rows × 242 columns


In [8]:
def buildFeatureAll(d, feature):
    prefix = '整体'
    group_by=d.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                      '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],as_index=False)
    agg_param = ['sum', 'count', 'max', 'min', 'median', 'mean', 'std', 'var']
    bill_beforeloan_agg = group_by.agg(agg_param)
    bill_beforeloan_agg.columns = ['_'.join(x) for x in bill_beforeloan_agg.columns.ravel()]
    bill_beforeloan_agg = bill_beforeloan_agg.add_prefix(prefix).reset_index()
    feature=pd.merge(feature, bill_beforeloan_agg,how='left', on = "用户标识")
    #print(feature.shape)
    for stat in agg_param:
        feature[prefix+'上期还款金额'+stat+'与'+prefix+'上期账单金额'+stat+'差值']=feature[prefix+'上期还款金额_'+stat]-feature[prefix+'上期账单金额_'+stat]
        feature[prefix+'信用卡额度'+stat+'与'+prefix+'本期账单余额'+stat+'差值']=feature[prefix+'信用卡额度_'+stat]-feature[prefix+'本期账单余额_'+stat]
        feature[prefix+'可用余额'+stat+'与'+prefix+'预借现金额度'+stat+'差值']=feature[prefix+'可用余额_'+stat]-feature[prefix+'预借现金额度_'+stat]
        feature[prefix+'本期账单最低还款额'+stat+'与'+prefix+'循环利息'+stat+'之和']=feature[prefix+'本期账单最低还款额_'+stat]+feature[prefix+'循环利息_'+stat]
    #print(feature.shape)
    
    feature_basic = d
    feature_basic['相邻两期账单金额差'] = feature_basic['本期账单金额'] - feature_basic['上期账单金额']
    feature_basic['本期还款总额'] = feature_basic['上期账单金额'] - feature_basic['上期还款金额'] + feature_basic['本期账单金额'] - feature_basic['调整金额'] + feature_basic['循环利息']
    feature_basic['已经使用信用卡额度'] = feature_basic['信用卡额度'] - feature_basic['可用余额']
    gb = feature_basic.loc[:,['用户标识' ,'相邻两期账单金额差', '本期还款总额', '已经使用信用卡额度']].groupby(["用户标识"], as_index=False)
    feature_agg = gb.agg({'mean', 'var', 'max', 'min', 'median'}).add_prefix(prefix+"_").reset_index()
    feature=pd.merge(feature, feature_agg,how='left', on = "用户标识")
    #print(feature.shape)
    return feature#55596 rows × 242 columns

In [9]:
def buildFeatureAll2(d, feature):
    #整体账单去重统计特征
    #按用户标识\时间\银行标识汇总统计（去重）
    prefix = '整体去重'
    data=d.loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
                  ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
                  ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

    group_by=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                   '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

    agg_param = ['sum', 'count', 'max', 'min', 'median', 'mean', 'std', 'var']
    bill_beforeloan_agg = group_by.agg(agg_param)
    bill_beforeloan_agg.columns = ['_'.join(x) for x in bill_beforeloan_agg.columns.ravel()]
    bill_beforeloan_agg = bill_beforeloan_agg.add_prefix(prefix).reset_index()
    feature=pd.merge(feature, bill_beforeloan_agg,how='left', on = "用户标识")
    #print(feature.shape)
    for stat in agg_param:
        feature[prefix+'上期还款金额'+stat+'与'+prefix+'上期账单金额'+stat+'差值']=feature[prefix+'上期还款金额_'+stat]-feature[prefix+'上期账单金额_'+stat]
        feature[prefix+'信用卡额度'+stat+'与'+prefix+'本期账单余额'+stat+'差值']=feature[prefix+'信用卡额度_'+stat]-feature[prefix+'本期账单余额_'+stat]
        feature[prefix+'可用余额'+stat+'与'+prefix+'预借现金额度'+stat+'差值']=feature[prefix+'可用余额_'+stat]-feature[prefix+'预借现金额度_'+stat]
        feature[prefix+'本期账单最低还款额'+stat+'与'+prefix+'循环利息'+stat+'之和']=feature[prefix+'本期账单最低还款额_'+stat]+feature[prefix+'循环利息_'+stat]
    #print(feature.shape)
    return feature#55596 rows × 241 columns

In [10]:
def buildFeatureBasic(d, feature):
    #----------------------------------------放款前特征统计------------------------------------------#

    #统计放款前用户上期账单金额值总额以及用户账单金额为负数的情况统计
    gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)['上期账单金额']
    x1=gb.apply(lambda x:x.where(x<0).count())
    x2=gb.apply(lambda x:x.where(x==0.000000).count())
    x=gb.agg({'放款前账单金额统计' : 'sum'})
    x['放款前账单金额为负数']=x1
    x['放款前账单金额为零']=x2

    feature=pd.merge(feature, x,how='left', on = "用户标识")

    #统计放款前用户上期还款金额值总额以及用户还款金额为负数(零)的情况统计
    gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)['上期还款金额']
    x1=gb.apply(lambda x:x.where(x<0).count())
    x2=gb.apply(lambda x:x.where(x==0.000000).count())
    x=gb.agg({'放款前还款金额统计' : 'sum'})
    x['放款前还款金额为负数']=x1
    x['放款前还款金额为零']=x2

    feature=pd.merge(feature, x,how='left', on = "用户标识")
    feature['放款前账单还款差额']=feature['放款前账单金额统计']-feature['放款前还款金额统计']

    #删除0和负等异常值
    d1=d[(d['上期账单金额']<=0)].index.tolist()
    d=d.drop(d1,axis=0)
    d2=d[(d['上期还款金额']<=0)].index.tolist()
    d=d.drop(d2,axis=0)
    #删除0和负等异常值后的d共1625621行

    #按用户标识\时间\银行标识汇总统计

    gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识","时间","银行标识"],as_index=False)
    x1=gb['上期账单金额'].agg({'放款前该用户该银行上月账单金额总计' : 'sum','放款前该用户该银行上月账单金额最大值' : 'max'})
    x2=gb['上期还款金额'].agg({'放款前该用户该银行上月还款金额总计' : 'sum','放款前该用户该银行还款金额最大值' : 'max'})
    x3=gb['消费笔数'].agg({'用户放款前消费笔数最大值' : 'max'})
    x4=gb['循环利息'].agg({'用户放款前循环利息最大值' : 'max'})

    gb1=x1.groupby(["用户标识"],as_index=False)
    gb2=x2.groupby(["用户标识"],as_index=False)
    gb3=x3.groupby(["用户标识"],as_index=False)
    gb4=x4.groupby(["用户标识"],as_index=False)

    x11=gb1['放款前该用户该银行上月账单金额总计'].agg({'放款前该用户账单金额汇总(去重)' : 'sum','放款前该用户账单数(去重)' : 'count'})
    x12=gb1['放款前该用户该银行上月账单金额最大值'].agg({'放款前该用户账单金额最大值汇总(去重)' : 'sum'})

    x21=gb2['放款前该用户该银行上月还款金额总计'].agg({'放款前该用户账单还款金额汇总(去重)' : 'sum'})
    x22=gb2['放款前该用户该银行还款金额最大值'].agg({'放款前该用户账单还款金额最大值汇总(去重)' : 'sum'})

    x31=gb3['用户放款前消费笔数最大值'].agg({'用户放款前消费笔数(去重)' : 'sum'})
    x41=gb4['用户放款前循环利息最大值'].agg({'用户放款前循环利息(去重)' : 'sum'})

    feature=pd.merge(feature, x11,how='left', on = "用户标识")
    feature=pd.merge(feature, x12,how='left', on = "用户标识")
    feature=pd.merge(feature, x21,how='left', on = "用户标识")
    feature=pd.merge(feature, x22,how='left', on = "用户标识")
    feature=pd.merge(feature, x31,how='left', on = "用户标识")
    feature=pd.merge(feature, x41,how='left', on = "用户标识")

    x=pd.merge(x1, x2,how='inner')
    gb3=x[(x['放款前该用户该银行上月账单金额最大值']>x['放款前该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
    gb4=x[(x['放款前该用户该银行上月账单金额最大值']==x['放款前该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
    gb5=x[(x['放款前该用户该银行上月账单金额最大值']<x['放款前该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)

    x31=gb3['用户标识'].agg({'放款前账单大于还款计数(去重)' : 'count'})
    x32=gb4['用户标识'].agg({'放款前账单等于还款计数(去重)' : 'count'})
    x33=gb5['用户标识'].agg({'放款前账单小于还款计数(去重)' : 'count'})

    feature=pd.merge(feature, x31,how='left', on = "用户标识")
    feature=pd.merge(feature, x32,how='left', on = "用户标识")
    feature=pd.merge(feature, x33,how='left', on = "用户标识")

    feature['放款前账单汇总还款差额(去重)']=feature['放款前该用户账单金额汇总(去重)']-feature['放款前该用户账单还款金额汇总(去重)']
    feature['放款前账单最大值还款差额(去重)']=feature['放款前该用户账单金额最大值汇总(去重)']-feature['放款前该用户账单还款金额最大值汇总(去重)']

    #统计放款前用户消费笔数，循环利息总计
    gb=d[(d['时间']<=d['放款时间'])].groupby(["用户标识"],as_index=False)
    x1=gb['消费笔数'].agg({'用户放款前消费笔数' : 'sum'})
    x2=gb['循环利息'].agg({'用户放款前循环利息' : 'sum'})
    x3=gb['信用卡额度'].agg({'用户放款前信用卡额度最大值' : 'max'})

    feature=pd.merge(feature, x1,how='left', on = "用户标识")
    feature=pd.merge(feature, x2,how='left', on = "用户标识")
    feature=pd.merge(feature, x3,how='left', on = "用户标识")
    return feature

def buildFeatureBasic2(d, feature):
    #----------------------------------------放款后特征统计------------------------------------------#
    #统计放款后用户上期账单金额值总额以及用户账单金额为负数的情况统计
    gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)['上期账单金额']
    x1=gb.apply(lambda x:x.where(x<0).count())
    x2=gb.apply(lambda x:x.where(x==0.000000).count())
    x=gb.agg({'放款后账单金额统计' : 'sum'})
    x['放款后账单金额为负数']=x1
    x['放款后账单金额为零']=x2

    feature=pd.merge(feature, x,how='left', on = "用户标识")

    #统计放款后用户上期还款金额值总额以及用户还款金额为负数(零)的情况统计
    gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)['上期还款金额']
    x1=gb.apply(lambda x:x.where(x<0).count())
    x2=gb.apply(lambda x:x.where(x==0.000000).count())
    x=gb.agg({'放款后还款金额统计' : 'sum'})
    x['放款后还款金额为负数']=x1
    x['放款后还款金额为零']=x2

    feature=pd.merge(feature, x,how='left', on = "用户标识")

    feature['放款后账单还款差额']=feature['放款后账单金额统计']-feature['放款后还款金额统计']

    #删除0和负等异常值
    d1=d[(d['上期账单金额']<=0)].index.tolist()
    d=d.drop(d1,axis=0)
    d2=d[(d['上期还款金额']<=0)].index.tolist()
    d=d.drop(d2,axis=0)
    #删除0和负等异常值后的d共1625621行

    #按用户标识\时间\银行标识汇总统计
    gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识","时间","银行标识"],as_index=False)
    x1=gb['上期账单金额'].agg({'放款后该用户该银行上月账单金额总计' : 'sum','放款后该用户该银行上月账单金额最大值' : 'max'})
    x2=gb['上期还款金额'].agg({'放款后该用户该银行上月还款金额总计' : 'sum','放款后该用户该银行还款金额最大值' : 'max'})
    x3=gb['消费笔数'].agg({'用户放款后消费笔数最大值' : 'max'})
    x4=gb['循环利息'].agg({'用户放款后循环利息最大值' : 'max'})

    gb1=x1.groupby(["用户标识"],as_index=False)
    gb2=x2.groupby(["用户标识"],as_index=False)
    gb3=x3.groupby(["用户标识"],as_index=False)
    gb4=x4.groupby(["用户标识"],as_index=False)

    x11=gb1['放款后该用户该银行上月账单金额总计'].agg({'放款后该用户账单金额汇总(去重)' : 'sum','放款后该用户账单数(去重)' : 'count'})
    x12=gb1['放款后该用户该银行上月账单金额最大值'].agg({'放款后该用户账单金额最大值汇总(去重)' : 'sum'})

    x21=gb2['放款后该用户该银行上月还款金额总计'].agg({'放款后该用户账单还款金额汇总(去重)' : 'sum'})
    x22=gb2['放款后该用户该银行还款金额最大值'].agg({'放款后该用户账单还款金额最大值汇总(去重)' : 'sum'})

    x31=gb3['用户放款后消费笔数最大值'].agg({'用户放款后消费笔数(去重)' : 'sum'})
    x41=gb4['用户放款后循环利息最大值'].agg({'用户放款后循环利息(去重)' : 'sum'})

    feature=pd.merge(feature, x11,how='left', on = "用户标识")
    feature=pd.merge(feature, x12,how='left', on = "用户标识")
    feature=pd.merge(feature, x21,how='left', on = "用户标识")
    feature=pd.merge(feature, x22,how='left', on = "用户标识")
    feature=pd.merge(feature, x31,how='left', on = "用户标识")
    feature=pd.merge(feature, x41,how='left', on = "用户标识")

    x=pd.merge(x1, x2,how='inner')
    gb3=x[(x['放款后该用户该银行上月账单金额最大值']>x['放款后该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
    gb4=x[(x['放款后该用户该银行上月账单金额最大值']==x['放款后该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)
    gb5=x[(x['放款后该用户该银行上月账单金额最大值']<x['放款后该用户该银行还款金额最大值'])].groupby(["用户标识"],as_index=False)

    x31=gb3['用户标识'].agg({'放款后账单大于还款计数(去重)' : 'count'})
    x32=gb4['用户标识'].agg({'放款后账单等于还款计数(去重)' : 'count'})
    x33=gb5['用户标识'].agg({'放款后账单小于还款计数(去重)' : 'count'})

    feature=pd.merge(feature, x31,how='left', on = "用户标识")
    feature=pd.merge(feature, x32,how='left', on = "用户标识")
    feature=pd.merge(feature, x33,how='left', on = "用户标识")

    feature['放款后账单汇总还款差额(去重)']=feature['放款后该用户账单金额汇总(去重)']-feature['放款后该用户账单还款金额汇总(去重)']
    feature['放款后账单最大值还款差额(去重)']=feature['放款后该用户账单金额最大值汇总(去重)']-feature['放款后该用户账单还款金额最大值汇总(去重)']

    #统计放款前用户消费笔数，循环利息总计
    gb=d[(d['时间']>d['放款时间'])].groupby(["用户标识"],as_index=False)
    x1=gb['消费笔数'].agg({'用户放款后消费笔数' : 'sum'})
    x2=gb['循环利息'].agg({'用户放款后循环利息' : 'sum'})
    x3=gb['信用卡额度'].agg({'用户放款后信用卡额度最大值' : 'max'})

    feature=pd.merge(feature, x1,how='left', on = "用户标识")
    feature=pd.merge(feature, x2,how='left', on = "用户标识")
    feature=pd.merge(feature, x3,how='left', on = "用户标识")
    return feature

def buildFeatureBasic3(d, feature):
    #----------------------------------------总体特征统计------------------------------------------#
    #爆卡指的是本期账单余额大于信用卡额度
    gb=d[(d['信用卡额度']<d['本期账单余额'])].groupby(["用户标识"],as_index=False)
    x1=gb['时间'].apply(lambda x:np.unique(x).size)
    x=gb['时间'].agg({'爆卡次数' : 'count'})
    x['爆卡次数(去重)']=x1
    feature=pd.merge(feature, x,how='left', on = "用户标识")

    #用户持卡数
    gb=d.groupby(["用户标识"],as_index=False)
    x=gb['银行标识'].apply(lambda x:np.unique(x).size)
    x1=gb['银行标识'].agg({'用户银行卡账单计数' : 'count'})
    x1['用户持卡数']=x
    feature=pd.merge(feature,x1,how='left', on = "用户标识")
    return feature

def buildFeatureBasic4(d, feature):
    #----------------------------------------老段子的特征------------------------------------------#

    #老段子的特征...神了个奇
    t1=d[(d['时间']>d['放款时间'])].groupby("用户标识",as_index=False)
    t2=d[(d['时间']>d['放款时间']+1)].groupby("用户标识",as_index=False)
    t3=d[(d['时间']>d['放款时间']+2)].groupby("用户标识",as_index=False)

    x=t1['时间'].apply(lambda x:np.unique(x).size)
    x1=t1['时间'].agg({'老段子特征1' : 'count'})
    x1['x1']=x

    x=t2['时间'].apply(lambda x:np.unique(x).size)
    x2=t2['时间'].agg({'老段子特征2' : 'count'})
    x2['x2']=x

    x=t3['时间'].apply(lambda x:np.unique(x).size)
    x3=t3['时间'].agg({'老段子特征3' : 'count'})
    x3['x3']=x

    t=feature[['用户标识']]
    t=pd.merge(t,x1,how='left',on = "用户标识")
    t=pd.merge(t,x2,how='left',on = "用户标识")
    t=pd.merge(t,x3,how='left',on = "用户标识")
    t=t[['用户标识','x1','x2','x3','老段子特征1','老段子特征2','老段子特征3']]

    feature=pd.merge(feature, t,how='left', on = "用户标识")

    feature['老段子特征x']=(feature['x1']+1)*(feature['x2']+1)*(feature['x3']+1)
    #from sklearn.preprocessing import MinMaxScaler
    #feature.老段子特征x = MinMaxScaler().fit_transform(feature.老段子特征x)
    return feature


    

In [11]:
def constructBillDeatilFeature(loantime_file, bill_detail_file, 
                               output_file_time, output_file_notime, output_file_all, output_file_basic):
    loantime = pd.read_csv(loantime_file,header=0,names=['用户标识','放款时间'])
    loantime['放款时间']=loantime['放款时间']//86400

    bill_detail=pd.read_csv(bill_detail_file,
                     header=0,
                     names=['用户标识','时间','银行标识','上期账单金额','上期还款金额','信用卡额度',
                           '本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
                          '循环利息','可用余额','预借现金额度','还款状态']).drop_duplicates()
    bill_detail['时间']=bill_detail['时间']//86400

    bill_detail = pd.merge(bill_detail, loantime,how='inner', on = "用户标识")
    
    # save features with 时间>0
    d=bill_detail[(bill_detail['时间']>0)]
    features = buildFeatureBeforeLoan(d, loantime)
    print('buildFeatureBeforeLoan：')
    print(features.shape)
    features = buildFeatureBeforeLoan2(d, features)
    print('buildFeatureBeforeLoan2：')
    print(features.shape)
    
    features = buildFeatureAfterLoan(d, features)
    print('buildFeatureAfterLoan：')
    print(features.shape)
    features = buildFeatureAfterLoan2(d, features)
    print('buildFeatureAfterLoan2：')
    print(features.shape)
    print('save to csv:' + output_file_time)
    features.to_csv(output_file_time, index=None, encoding=FILE_ENCODER)
    
    # save features with 时间==0
    d=bill_detail[(bill_detail['时间']==0)]
    #print(d)
    features_notime = buildFeatureNotime(d, loantime[['用户标识']])
    print('buildFeatureNotime:')
    print(features_notime.shape)
    features_notime = buildFeatureNotime2(d, features_notime)
    print('buildFeatureNotime2:')
    print(features_notime.shape)
    print('save to csv:' + output_file_notime)
    features_notime.to_csv(output_file_notime, index=None, encoding=FILE_ENCODER)
    
    #整体
    features_all = buildFeatureAll(bill_detail, loantime[['用户标识']])
    print('buildFeatureAll:')
    print(features_all.shape)
    features_all = buildFeatureAll2(bill_detail, features_all)
    print('buildFeatureAll2:')
    print(features_all.shape)
    print('save to csv:' + output_file_all)
    features_all.to_csv(output_file_all, index=None, encoding=FILE_ENCODER)
    
    #basic
    features_basic = buildFeatureBasic(bill_detail, loantime)
    print('buildFeatureBasic:')
    print(features_basic.shape)
    features_basic = buildFeatureBasic2(bill_detail, features_basic)
    print('buildFeatureBasic2:')
    print(features_basic.shape)
    features_basic = buildFeatureBasic3(bill_detail, features_basic)
    print('buildFeatureBasic3:')
    print(features_basic.shape)
    features_basic = buildFeatureBasic4(bill_detail, features_basic)
    print('buildFeatureBasic4:')
    print(features_basic.shape)
    print('save to csv:' + output_file_basic)    
    
    features_basic.to_csv(output_file_basic, index=None, encoding=FILE_ENCODER)


In [12]:
def constructBillDeatilFeature2(loantime_file, bill_detail_file, 
                               output_file_time):
    loantime = pd.read_csv(loantime_file,header=0,names=['用户标识','放款时间'])
    loantime['放款时间']=loantime['放款时间']//86400

    bill_detail=pd.read_csv(bill_detail_file,
                     header=0,
                     names=['用户标识','时间','银行标识','上期账单金额','上期还款金额','信用卡额度',
                           '本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
                          '循环利息','可用余额','预借现金额度','还款状态']).drop_duplicates()
    bill_detail['时间']=bill_detail['时间']//86400

    bill_detail = pd.merge(bill_detail, loantime,how='inner', on = "用户标识")

    #groupby_condition_id_bank = ['用户标识','银行标识']
    groupby_condition_id = ['用户标识']
    agg_param = ['mean', 'var', 'max', 'min', 'median']

    #------------ ['用户标识','时间','银行标识']--------------
    bill_detail['上期未还款金额'] = bill_detail['上期账单金额'] - bill_detail['上期还款金额']
    bill_detail['相邻两期账单金额差'] = bill_detail['本期账单金额'] - bill_detail['上期账单金额']
    bill_detail['本期还款总额'] = bill_detail['上期账单金额'] - bill_detail['上期还款金额'] + bill_detail['本期账单金额'] - bill_detail['调整金额'] + bill_detail['循环利息']
    bill_detail['已经使用信用卡额度'] = bill_detail['信用卡额度'] - bill_detail['可用余额']
    #每张账单上每笔消费金额=本期账单金额/消费笔数
    bill_detail['每张账单上每笔消费金额'] = bill_detail['本期账单金额'] / bill_detail['消费笔数']
    bill_detail['是否逾期'] = bill_detail['上期未还款金额']>0
    

    # ['用户标识','银行标识']
    bill_detail_stat = bill_detail[:]
    bill_detail_stat = bill_detail_stat.drop(['银行标识','是否逾期'],axis=1)
    bill_detail_id_agg = bill_detail_stat.groupby(groupby_condition_id, as_index=False).agg(agg_param)
    bill_detail_id_agg.columns = ['_'.join(x) for x in bill_detail_id_agg.columns.ravel()]
    bill_detail_id_agg = bill_detail_id_agg.reset_index()
    #-----------------['用户标识']----------------
    ##逾期次数
    bill_detail_id_expire = bill_detail.loc[:,['用户标识','是否逾期','银行标识','信用卡额度']].groupby(groupby_condition_id,
                                                                        as_index=False).agg({'是否逾期':'sum'}).rename(index=str,
                                                                                                                   columns={'是否逾期':'逾期次数'})
    bill_detail_id_agg = pd.merge(bill_detail_id_agg, bill_detail_id_expire,how='left', on = "用户标识")
    ##信用卡张数
    bill_detail_bankcount = bill_detail.loc[:,['用户标识','银行标识']].drop_duplicates().groupby(groupby_condition_id, as_index=False).agg('count').rename(index=str, columns={'银行标识':'信用卡张数'})
    bill_detail_id_agg = pd.merge(bill_detail_id_agg, bill_detail_bankcount,how='left', on = "用户标识")
    ##所有信用卡的总额度=sum(distinct(银行id,信用卡额度))
    bill_detail_accountsum = bill_detail.loc[:,['用户标识','银行标识','信用卡额度']].drop_duplicates().groupby(groupby_condition_id, 
                                                                                                  as_index=False).agg({'信用卡额度':'sum'}
                                                                                                                     ).rename(index=str, columns={'信用卡额度':'所有信用卡的总额度'})
    bill_detail_id_agg = pd.merge(bill_detail_id_agg, bill_detail_accountsum,how='left', on = "用户标识")
    ## 平均每张信用卡的额度=所有信用卡的总额度/信用卡张数
    bill_detail_id_agg['平均每张信用卡的额度'] = bill_detail_id_agg['所有信用卡的总额度'] / bill_detail_id_agg['信用卡张数']
    ## 信用卡最高额度=max(信用卡额度) 信用卡最低额度=min(信用卡额度) 
    ## 信用卡平均消费最高值=max(每张账单上每笔消费金额)？
    ## 信用卡平均消费最低值=min(每张账单上每笔消费金额)？
    ## 信用卡平均最高消费次数=max(mean(消费笔数)) TO DO
    ## 信用卡平均最低消费次数=min(mean(消费笔数)) TO DO
    
    
    ## 计算逾期天数
    temp = bill_detail.sort_values(by=['用户标识', '时间'], ascending = True)
    df = pd.DataFrame(columns = ['用户标识','逾期天数'])
    first_index = 0
    overdue_days = 0
    first_overdue_day = 0
    last_overdue_day = 0
    max_overdue_days = 0
    count = 0
    # for index, row in temp.iterrows():
    #     count=count+1
    # print(count)

    for index, row in temp.iterrows():
        if first_index == 0:
            first_index = row['用户标识']

        #print(first_index, ',', first_overdue_day,',',last_overdue_day)
        if first_index != 0 and first_index != row['用户标识']:
            if last_overdue_day - first_overdue_day>max_overdue_days:
                max_overdue_days = last_overdue_day - first_overdue_day
                first_overdue_day = 0
                last_overdue_day = 0
            #print(first_index, ',', max_overdue_days)
            df.loc[count] = [first_index, max_overdue_days]
            count = count + 1
            first_index = row['用户标识']
            overdue_days = 0
            first_overdue_day = 0
            last_overdue_day = 0
            max_overdue_days = 0


        if row['是否逾期'] == True:
            if first_overdue_day == 0:
                first_overdue_day = row['时间']
            else:
                last_overdue_day = row['时间']
        else:
            if last_overdue_day - first_overdue_day>max_overdue_days:
                max_overdue_days = last_overdue_day - first_overdue_day
                #print(first_index, ',', max_overdue_days)
                first_overdue_day = 0
                last_overdue_day = 0
    # END
    bill_detail_id_agg = pd.merge(bill_detail_id_agg, df, how='left', on ='用户标识')
    print(bill_detail_id_agg.shape)
    bill_detail_id_agg.to_csv(output_file_time, index=None, encoding=FILE_ENCODER)


In [13]:
print("begin")
constructBillDeatilFeature2(LOANTIME_TRAIN_FILE, 
                           BILL_DETAIL_TRAIN_FILE,
                          "../feature/bill_detail_feature2_train_20181106.csv")
constructBillDeatilFeature2(LOANTIME_TEST_FILE, 
                           BILL_DETAIL_TEST_FILE,
                          "../feature/bill_detail_feature2_test_20181106.csv")
print("end")


begin


FileNotFoundError: File b'../train/loantime_train.csv' does not exist

In [14]:
print("begin")
# bill_detail
# 有时间记录的账单特征存储在bill_detail_xxx_time20181106，没有时间记录的账单特征存储在bill_detail_xxx_notime20181106
constructBillDeatilFeature(LOANTIME_TRAIN_FILE, 
                           BILL_DETAIL_TRAIN_FILE,
                          "../feature/bill_detail_train_time20181106.csv",
                          "../feature/bill_detail_train_notime20181106.csv",
                          "../feature/bill_detail_train_all20181106.csv",
                          "../feature/bill_detail_train_basic20181106.csv")
constructBillDeatilFeature(LOANTIME_TEST_FILE, 
                           BILL_DETAIL_TEST_FILE,
                          "../feature/bill_detail_test_time20181106.csv",
                          "../feature/bill_detail_test_notime20181106.csv",
                          "../feature/bill_detail_test_all20181106.csv",
                          "../feature/bill_detail_test_basic20181106.csv")

print("end")


begin


FileNotFoundError: File b'../train/loantime_train.csv' does not exist